In [20]:
import glob
import cv2
import numpy as np
import pdb
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch

## Calibration

In [21]:
valid_id = list(range(2, 4)) + list(range(6, 21))
imageList = []
for i in valid_id:
    imageList.append("camera_cal/calibration" + str(i) + ".jpg")
# print(imageList)
objpoints = []
imgpoints = []
nx = 9
ny = 6
objp = np.zeros((nx*ny, 3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)

for filename in imageList:
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
    if ret == True:
        img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
        imgpoints.append(corners)
        objpoints.append(objp)


gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)


In [22]:
img = cv2.imread(imageList[0])
plt.figure()
plt.imshow(img)
dst = cv2.undistort(img, mtx, dist, None, mtx)
plt.figure()
plt.imshow(dst)
plt.imsave("calibrated.png", dst)
plt.show()

KeyboardInterrupt: 

## Perspective transform

In [ ]:
img = mpimg.imread("test_images/straight_lines1.jpg")
img = cv2.undistort(img, mtx, dist, None, mtx)
plt.imshow(img)

src = np.float32(
    [[710, 464],
     [1055, 689],
     [248, 689],
     [574, 464]])

for i in range(4):
    plt.plot(src[i, 0], src[i, 1], '.')
    
dst = np.float32(
    [[1055, 0],
     [1055, 720],
     [248, 720],
     [248, 0]])

plt.savefig("Perspective_points.png")

M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)
warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))
plt.figure()
plt.imshow(warped)
plt.imsave("warped.png", warped)
plt.show()

## Compute feature

In [ ]:
def get_feature(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    thresh_min = 20
    thresh_max = 100
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    s_thresh_min = 170
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    

    return combined_binary, color_binary

def visualize_feature(combined_binary, color_binary):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.set_title('Stacked thresholds')
    ax1.imshow(color_binary)

    ax2.set_title('Combined S channel and gradient thresholds')
    ax2.imshow(combined_binary, cmap='gray')
    plt.show()

img = mpimg.imread("test_images/test2.jpg")
img = cv2.undistort(img, mtx, dist, None, mtx)
combined_binary, color_binary = get_feature(img)
visualize_feature(combined_binary, color_binary)
plt.imsave("color_binary.png", color_binary)
plt.imsave("combined_binary.png", combined_binary)


## fitting

In [ ]:
def fit(binary_warped, visualize = False):
    histogram = np.sum(binary_warped[binary_warped.shape[0]//4:,:], axis=0)
#     plt.plot(histogram)
#     plt.figure()
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    nwindows = 9
    window_height = np.int(binary_warped.shape[0]/nwindows)

    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 80
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2)
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2)
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each


    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    if visualize:
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
        plt.imshow(out_img)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
        plt.savefig("out_img.png", dpi = 100)
    
    alpha = np.zeros((out_img.shape[0], out_img.shape[1], 3))
    overlay_img = np.zeros_like(out_img)
    overlay_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    overlay_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    alpha[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [1, 1, 1]
    alpha[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [1, 1, 1]
    for y in range(binary_warped.shape[0]):
        overlay_img[y, int(left_fitx[y]):int(right_fitx[y])] = [0, 255, 0]
        alpha[y, int(left_fitx[y]):int(right_fitx[y])] = [1, 1, 1]
        
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    y_eval = binary_warped.shape[0]
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    # compute the combined lane and overall curvature
    # Make the left and right lane overlap together and compute polynomial again
    totaly = np.concatenate([lefty, righty])
    
    lr_dist = rightx_base - leftx_base
    
    totalx = np.concatenate([leftx + lr_dist/2, rightx - lr_dist/2])
    total_fit = np.polyfit(totaly*ym_per_pix, totalx*xm_per_pix, 2)
    tota_fitx = total_fit[0]*(ploty*ym_per_pix)**2 + total_fit[1]*(ploty*ym_per_pix) + total_fit[2]
    total_curverad = ((1 + (2*total_fit[0]*y_eval*ym_per_pix + total_fit[1])**2)**1.5) / np.absolute(2*total_fit[0])
    
#     plt.plot(tota_fitx/xm_per_pix, ploty, color='yellow')
#     print(total_curverad, left_curverad, right_curverad)
    
    offset = ((rightx_base + leftx_base)/2 - midpoint)*xm_per_pix
    return total_curverad, offset, overlay_img, alpha

warped_binary = cv2.warpPerspective(combined_binary, M, (combined_binary.shape[1], combined_binary.shape[0]))
plt.imsave("warped_binary.png", warped_binary, cmap = 'gray')
total_curverad, offset, overlay_img, alpha = fit(warped_binary, True)

overlay_img = cv2.warpPerspective(overlay_img, Minv, (overlay_img.shape[1], overlay_img.shape[0]))
alpha = cv2.warpPerspective(alpha, Minv, (alpha.shape[1], alpha.shape[0]))
plt.figure()
plt.imshow(overlay_img)
plt.show()

## Visualize result

In [ ]:
def visualize_result(img, overlay_img, alpha, curvature, offset):
    result = img*(1 - alpha) + (img*0.5 + overlay_img*0.5)*alpha
    cv2.putText(result, "Curvature: {0:.3f} m".format(curvature), (100, 100), fontFace = cv2.FONT_HERSHEY_SIMPLEX, thickness = 3, fontScale = 2, color = (255, 255, 255))
    cv2.putText(result, "Vehicle is {0:.3f} m left of center".format(offset), (100, 200), fontFace = cv2.FONT_HERSHEY_SIMPLEX, thickness = 3, fontScale = 2, color = (255, 255, 255))
    return result.astype(np.uint8)

result = visualize_result(img, overlay_img, alpha, total_curverad, offset)
plt.imshow(result)
plt.imsave("result.png", result)
plt.show()

## Test images

In [ ]:
img_list = glob.glob("test_images/*.jpg")
for img_name in img_list:
    img = mpimg.imread(img_name)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    combined_binary, color_binary = get_feature(img)
    warped_binary = cv2.warpPerspective(combined_binary, M, (combined_binary.shape[1], combined_binary.shape[0]))
    total_curverad, offset, overlay_img, alpha = fit(warped_binary)
    overlay_img = cv2.warpPerspective(overlay_img, Minv, (overlay_img.shape[1], overlay_img.shape[0]))
    alpha = cv2.warpPerspective(alpha, Minv, (alpha.shape[1], alpha.shape[0]))
    result = visualize_result(img, overlay_img, alpha, total_curverad, offset)
    plt.figure()
    plt.imshow(result)
plt.show()    

## Test video

In [23]:
import imageio
filename = 'challenge_video.mp4'
vid = imageio.get_reader(filename,  'ffmpeg')
fps = vid.get_meta_data()['fps']
writer = imageio.get_writer('project_video_out.mp4', fps=fps)

for i, img in enumerate(vid):
    print("frame", i)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    combined_binary, color_binary = get_feature(img)
    warped_binary = cv2.warpPerspective(combined_binary, M, (combined_binary.shape[1], combined_binary.shape[0]))
    total_curverad, offset, overlay_img, alpha = fit(warped_binary)
    overlay_img = cv2.warpPerspective(overlay_img, Minv, (overlay_img.shape[1], overlay_img.shape[0]))
    alpha = cv2.warpPerspective(alpha, Minv, (alpha.shape[1], alpha.shape[0]))
    result = visualize_result(img, overlay_img, alpha, total_curverad, offset)
    writer.append_data(result)
writer.close()

frame 0
frame 1
frame 2
frame 3
frame 4
frame 5
frame 6
frame 7
frame 8
frame 9
frame 10
frame 11
frame 12
frame 13
frame 14
frame 15
frame 16
frame 17
frame 18
frame 19
frame 20
frame 21
frame 22
frame 23
frame 24
frame 25
frame 26
frame 27
frame 28
frame 29
frame 30
frame 31
frame 32
frame 33
frame 34
frame 35
frame 36
frame 37
frame 38
frame 39
frame 40
frame 41
frame 42
frame 43
frame 44
frame 45
frame 46
frame 47
frame 48
frame 49
frame 50
frame 51
frame 52
frame 53
frame 54
frame 55
frame 56
frame 57
frame 58
frame 59
frame 60
frame 61
frame 62
frame 63
frame 64
frame 65
frame 66
frame 67
frame 68
frame 69
frame 70
frame 71
frame 72
frame 73
frame 74
frame 75
frame 76
frame 77
frame 78
frame 79
frame 80
frame 81
frame 82
frame 83
frame 84
frame 85
frame 86
frame 87
frame 88
frame 89
frame 90
frame 91
frame 92
frame 93
frame 94
frame 95
frame 96
frame 97
frame 98
frame 99
frame 100
frame 101
frame 102
frame 103
frame 104
frame 105
frame 106
frame 107
frame 108
frame 109
frame 110
